In [ ]:
%pip install tensorflow
%pip install torch

In [ ]:
%pip install transformers

In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline

In [ ]:

data = pd.read_csv("data/tfidf.csv")

# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dslim/bert-large-NER")
model = AutoModelForTokenClassification.from_pretrained("dslim/bert-large-NER")

# Initialize the NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# Define a function to perform NER on a term
def get_ner_tags(term):
    ner_results = nlp(term)
    tags = [entity['entity_group'] for entity in ner_results]
    return ", ".join(tags)

# Apply the NER function to each term
data['NER Tags'] = data['Term'].apply(get_ner_tags)

data.to_csv("data/tfidf_NER_largemodel.csv", index=False)

In [ ]:
# Initialize the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")

# Initialize the NER pipeline
nlp = pipeline("ner", model=model, tokenizer=tokenizer, grouped_entities=True)

# Define a function to perform NER on a term
def get_ner_tags(term):
    ner_results = nlp(term)
    tags = [entity['entity_group'] for entity in ner_results]
    return ", ".join(tags)

# Apply the NER function to each term
data['NER Tags'] = data['Term'].apply(get_ner_tags)

data.to_csv("data/tfidf_NER1.csv", index=False)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Davlan/xlm-roberta-large-ner-hrl")
model = AutoModelForTokenClassification.from_pretrained("Davlan/xlm-roberta-large-ner-hrl")

Extract NER from OCR  directly

In [4]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import re

# Load the CSV file
df = pd.read_csv('data\ocr_texts_vip.csv')

# Initialize the BERT NER model and tokenizer
model_name = "dslim/bert-large-NER"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Function to extract text between "Subject:"/"Re:" and "CONFIDENTIAL"
def extract_relevant_text(text):
    match = re.search(r'Subject:.*?CONFIDENTIAL|Re:.*?CONFIDENTIAL', text, re.DOTALL)
    if match:
        relevant_text = match.group(0)
        # Remove 'CONFIDENTIAL' and the leading part up to "Subject:" or "Re:"
        relevant_text = re.sub(r'(Subject:|Re:)', '', relevant_text)
        relevant_text = relevant_text.replace('CONFIDENTIAL', '').strip()
        return relevant_text
    return ""

# Function to extract NER tags
def extract_ner_tags(text):
    ner_results = ner_pipeline(text)
    return ner_results

# Process each OCR text and extract NER tags
ner_data = []
for index, row in df.iterrows():
    doc_id = row['id']
    text = extract_relevant_text(row['text'])
    if text:
        ner_results = extract_ner_tags(text)
        for result in ner_results:
            word = result['word']
            entity = result['entity']
            ner_data.append({'id': doc_id, 'word': word, 'tag': entity})
            
# Convert the results to a DataFrame
ner_df = pd.DataFrame(ner_data)

# Save the results to a new CSV file
output_path = 'data/ner_output.csv'
ner_df.to_csv(output_path, index=False)


Some weights of the model checkpoint at dslim/bert-large-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


KeyboardInterrupt: 

In [ ]:
# Initialize the keyword extraction model and tokenizer
model_name = "transformer3/H2-keywordextractor"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

# Create a pipeline for keyword extraction
keyword_extraction_pipeline = pipeline("ner", model=model, tokenizer=tokenizer)

# Function to extract relevant text between "Subject:"/"Re:" and "CONFIDENTIAL"
def extract_relevant_text(text):
    match = re.search(r'(Subject:|Re:).*?(CONFIDENTIAL|$)', text, re.DOTALL)
    if match:
        relevant_text = match.group(0)
        # Remove 'CONFIDENTIAL' and the leading part up to "Subject:" or "Re:"
        relevant_text = re.sub(r'(Subject:|Re:)', '', relevant_text)
        relevant_text = relevant_text.replace('CONFIDENTIAL', '').strip()
        return relevant_text
    return ""

# Function to extract keywords
def extract_keywords(text):
    keyword_results = keyword_extraction_pipeline(text)
    return keyword_results

# Process each OCR text and extract keywords
keyword_data = []
for index, row in df.iterrows():
    doc_id = row['id']
    text = extract_relevant_text(row['text'])
    if text:
        keyword_results = extract_keywords(text)
        for result in keyword_results:
            word = result['word']
            entity = result['entity']
            keyword_data.append({'id': doc_id, 'word': word, 'tag': entity})

# Convert the results to a DataFrame
keyword_df = pd.DataFrame(keyword_data)

# Save the results to a new CSV file
output_path = 'data/keyword_output.csv'
keyword_df.to_csv(output_path, index=False)